# Label QM9 as Synthesizable or Not
QM9 is a well-known is a well-known dataset of molecular energies spanning most molecules with sizes between 1 and 9 heavy atoms. 
Given the small size of molecules, we assume that if it has not been reported in the literature and referenced in PubChem by now then it is not synthesizable.
Not a perfect assumption, but one we can use to get a rough idea about thermodynamic bounds of molecular synthesizability.

In [1]:
from emin.source import get_molecules_from_pubchem
from rdkit import Chem, RDLogger
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import requests
RDLogger.DisableLog('rdApp.*')

## Download QM9-G4MP2
We are going to use [a version of QM9 with energies computed at the high-accuracy, G4MP2 level](https://pubs.rsc.org/en/content/articlehtml/2019/sc/c9sc02834j) as a starting point.

In [2]:
%%time
qm9 = pd.read_json('https://github.com/globus-labs/g4mp2-atomization-energy/raw/master/data/output/g4mp2_data.json.gz', lines=True)
print(f'Loaded {len(qm9)} molecules')

Loaded 130258 molecules
CPU times: user 7.23 s, sys: 3.13 s, total: 10.4 s
Wall time: 14.9 s


Remove duplicates

In [3]:
qm9.sort_values('g4mp2_0k', ascending=True)
qm9.drop_duplicates('inchi_0', inplace=True, keep='first')
print(f'Trimmed down to {len(qm9)} unique molecules')

Trimmed down to 126405 unique molecules


## Compute Composition
Get the chemical composition of each, which we're going to use to find whether they are in PubChem

In [4]:
def get_composition(inchi: str):
    """Get the chemical composition from an InChI string
    
    Args:
        inchi: InChI string
    Returns:
        Chemical formula
    """
    return inchi.split("/")[1]

In [5]:
qm9['formula'] = qm9.inchi_1.apply(get_composition)

## Find in PubChem
PubChem has a [fantastic API](https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest) and we can use it to find which molecules held within QM9 are also in PubChem

Start by getting an InChI Key, which we can use to detect whether molecules are held in PubChem

In [6]:
%%time
qm9['inchi_key'] = qm9['inchi_1'].apply(Chem.MolFromInchi).apply(lambda x: Chem.MolToInchiKey(x) if x is not None else x)

CPU times: user 1min 29s, sys: 18.3 s, total: 1min 48s
Wall time: 2min 3s


Make a function that gets every InChI key, including charged species and stereoisomers

In [7]:
def get_inchi_keys(formula: str) -> set[str]:
    """Get the InChI keys of all molecules in PubChem with a certain formula
    
    Args:
        formula: Formula in question
    Returns:
        List of InChI keys
    """
    res = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastformula/{formula}/property/InChIKey/TXT')
    return set(res.content.decode().split())
get_inchi_keys('H2O')

{'RECVMTHOQWMYFX-UHFFFAOYSA-N',
 'UKEUXZKBWBZANT-UHFFFAOYSA-M',
 'XLYOFNOQVPJJNP-AKLPVKDBSA-N',
 'XLYOFNOQVPJJNP-BJUDXGSMSA-N',
 'XLYOFNOQVPJJNP-DYCDLGHISA-N',
 'XLYOFNOQVPJJNP-FTGQXOHASA-N',
 'XLYOFNOQVPJJNP-IGMARMGPSA-N',
 'XLYOFNOQVPJJNP-LAPWFLRPSA-N',
 'XLYOFNOQVPJJNP-MNYXATJNSA-N',
 'XLYOFNOQVPJJNP-NAQDNALJSA-N',
 'XLYOFNOQVPJJNP-NJFSPNSNSA-N',
 'XLYOFNOQVPJJNP-OIOBTWANSA-N',
 'XLYOFNOQVPJJNP-OUBTZVSYSA-N',
 'XLYOFNOQVPJJNP-PWCQTSIFSA-N',
 'XLYOFNOQVPJJNP-TXJFRLKBSA-N',
 'XLYOFNOQVPJJNP-UHFFFAOYSA-N',
 'XLYOFNOQVPJJNP-YOGHCVPTSA-N',
 'XLYOFNOQVPJJNP-YPZZEJLDSA-N',
 'XLYOFNOQVPJJNP-ZSJDYOACSA-N'}

Label whether every entry is in PubChem

In [8]:
qm9['in_pubchem'] = None

In [9]:
for formula, group in tqdm(qm9.groupby('formula')):
    known_inchi_keys = get_inchi_keys(formula)
    in_pubchem = group['inchi_key'].apply(known_inchi_keys.__contains__)
    qm9.loc[in_pubchem.index, 'in_pubchem'] = in_pubchem

100%|██████████| 702/702 [23:39<00:00,  2.02s/it]


In [10]:
frac = qm9['in_pubchem'].mean()
print(f'{frac*100:.1f}% of QM9 is in PubChem')

16.9% of QM9 is in PubChem


Save the content to disk

In [11]:
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)
qm9.to_json(data_dir / 'qm9.json.gz', lines=True, index=False, orient='records')